**This file uses Yolov3 to detect objects for each counting location**

In [ ]:
!pip install tensorflow==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow-gpu==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.9 MB 6.0 MB/s 
     |████████████████████████████████| 2.1 MB 32.9 MB/s 
     |████████████████████████████████| 11.6 MB 19.2 MB/s 
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=ea8882b2855f24b20b271cdbc48c01ca21f7da724efc8fc6a664a7f39934dfec
  Stored in directory: /root/.cache/pip/wheels/bd/ef/06/5d65f696360436c3a423020c4b7fd8c558c09ef264a0e6c575
Successfully built keras-resnet
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Success

In [ ]:
!pip install imageai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 160 kB 5.1 MB/s 


In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# 新段落

In [ ]:
from imageai.Detection import ObjectDetection
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('./drive/MyDrive/Streetview/central_2019_avg_newest.csv')
df.head()

,Location,Number of private cycles,Number of cycle hire bikes,Total cycles,lat,lng,geometry,LSOA11CD,LSOA11NM,accounting,airport,amusement park,aquarium,art gallery,atm,bakery,bank,bar,beauty salon,bicycle store,book store,bowling alley,bus station,cafe,campground,car dealer,car rental,car repair,car wash,casino,cemetery,church,city hall,clothing store,convenience store,courthouse,dentist,department store,doctor,drugstore,electrician,electronics store,embassy,fire station,florist,funeral home,furniture store,gas station,gym,hair care,hardware store,hindu temple,home goods store,hospital,insurance agency,jewelry store,laundry,lawyer,library,light rail station,liquor store,local government office,locksmith,lodging,meal delivery,meal takeaway,mosque,movie rental,movie theater,moving company,museum,night club,painter,park,parking,pet store,pharmacy,physiotherapist,plumber,police,post office,primary school,real estate agency,restaurant,roofing contractor,rv park,school,secondary school,shoe store,shopping mall,spa,stadium,storage,store,subway station,supermarket,synagogue,taxi stand,tourist attraction,train station,transit station,travel agency,university,veterinary care,zoo,person,bicycle,car,motorcycle,bus,train,truck,traffic light,fire hydrant,stop sign,parking meter,bench,dog
0,Albert Embankment (South of Salamanca Street),1470.375,162.250,1632.625,51.490324,-0.122632,POINT (-0.1226315 51.4903238),E01003109,Lambeth 003E,8,1,0,0,14,6,4,3,19,NaN,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,0,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,3,9,NaN,6,11,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,32,NaN,NaN,15,NaN,NaN,0,NaN,1,NaN,NaN,2,7,NaN,NaN,NaN,4,37,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aldgate High Street,1592.750,159.750,1752.500,51.514015,-0.075322,POINT (-0.0753219 51.5140145),E01000005,City of London 001E,55,2,0,0,25,20,9,56,60,NaN,NaN,NaN,NaN,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,26,NaN,NaN,NaN,NaN,1,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,5,10,NaN,8,29,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,60,NaN,NaN,20,NaN,NaN,11,NaN,0,NaN,NaN,9,18,NaN,NaN,NaN,7,49,NaN,10,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aldwych,1141.000,191.375,1332.375,51.513226,-0.117359,POINT (-0.1173591 51.5132258),E01004735,Westminster 018B,48,1,0,1,31,15,17,20,53,NaN,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,11,10,NaN,7,15,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,60,NaN,NaN,20,NaN,NaN,13,NaN,0,NaN,NaN,7,13,NaN,NaN,NaN,6,32,NaN,5,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Appold Street,764.125,84.750,848.875,51.520722,-0.081802,POINT (-0.0818023 51.5207216),E01032739,City of London 001F,60,3,0,0,49,23,10,56,60,NaN,NaN,NaN,NaN,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,33,NaN,NaN,NaN,NaN,0,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,7,11,NaN,3,31,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,60,NaN,NaN,13,NaN,NaN,13,NaN,0,NaN,NaN,9,22,NaN,NaN,NaN,6,59,NaN,6,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arundel Street,203.750,41.250,245.000,51.512529,-0.114368,POINT (-0.1143676 51.5125289),E01004735,Westminster 018B,50,2,0,1,16,15,12,23,40,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,11,10,NaN,5,18,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,60,NaN,NaN,17,NaN,NaN,7,NaN,0,NaN,NaN,6,15,NaN,NaN,NaN,3,29,NaN,4,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join('/content/drive/MyDrive/Streetview/' , "yolo.h5"))
detector.loadModel(detection_speed="fast")
custom_objects = detector.CustomObjects(person=True, bicycle=True,car=True,motorcycle=True,bus=True,train=True,truck=True,traffic_light=True,fire_hydrant=True,stop_sign=True,parking_meter=True,bench=True,dog=True)

In [ ]:
column_name = ['person','bicycle','car','motorcycle','bus', 'train', 'truck','traffic light', 'fire hydrant', 'stop sign',
'parking meter',  'bench','dog']
save_count = 0

for index, row in df.iterrows():
  print(list(df.loc[index,column_name]))
  if np.isnan(list(df.loc[index,column_name])).all():
    execution_path1 = '/content/drive/MyDrive/Streetview/Central_GSV_result'
    detect_list = []
    print('===========' + str(save_count) + '  ' + row['Location'] + '===========')
    for i in ['_0.jpg','_90.jpg','_180.jpg','_270.jpg']:
      print('----' + i + '-----')
      execution_path2 = '/content/drive/MyDrive/Streetview/Central_GSV_data/' 
      name = str(row['Location']) + str(i)
      detections = detector.detectCustomObjectsFromImage(custom_objects=custom_objects, input_image=os.path.join(execution_path2 , name), output_image_path=os.path.join(execution_path1 , 'result_'+name), minimum_percentage_probability=30)
      for eachObject in detections:
          print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
          if np.math.isnan(df.loc[index, str(eachObject["name"])]):
            df.loc[index, str(eachObject["name"])] = 1
            print('=', 1)
          else:
            df.loc[index, str(eachObject["name"])] = df.loc[index, str(eachObject["name"])] + 1
            print('+', df.loc[index, str(eachObject["name"])])
    save_count = save_count + 1
    if save_count == 10:
      df.to_csv('./drive/MyDrive/Streetview/central_2019_avg_newest.csv', index=False)
      save_count = 0
      print('SAVE')
  

  

[1.0, nan, 1.0, nan, nan, nan, 1.0, nan, nan, nan, nan, nan, nan]
[14.0, 1.0, 16.0, nan, 1.0, nan, nan, 1.0, nan, nan, nan, nan, nan]
[21.0, nan, 15.0, nan, 4.0, nan, 3.0, 3.0, nan, nan, nan, nan, nan]
[nan, 3.0, 5.0, nan, nan, nan, nan, 3.0, nan, nan, nan, nan, nan]
[16.0, nan, 19.0, 1.0, nan, nan, 5.0, 3.0, nan, nan, nan, nan, nan]
[nan, nan, 23.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[2.0, 1.0, 6.0, nan, nan, nan, nan, nan, 1.0, nan, nan, nan, nan]
[18.0, 1.0, 27.0, 3.0, nan, nan, nan, nan, nan, 1.0, nan, nan, nan]
[14.0, nan, 14.0, nan, nan, nan, 2.0, nan, nan, nan, nan, nan, nan]
[6.0, nan, 8.0, 1.0, nan, nan, 1.0, 1.0, nan, nan, nan, nan, nan]
[1.0, nan, 6.0, nan, nan, nan, 1.0, nan, nan, 1.0, nan, nan, nan]
[1.0, nan, 2.0, nan, nan, nan, nan, nan, nan, nan, nan, 1.0, nan]
[28.0, nan, 3.0, nan, 1.0, nan, nan, 3.0, nan, nan, nan, nan, nan]
[1.0, 1.0, nan, nan, nan, nan, 3.0, nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: 'detectCustomObjectsFromImage()' function has been deprecated and will be removed in future versions of ImageAI. 
 Kindly use 'detectObjectsFromImage()' 
  from ipykernel import kernelapp as app


流式输出内容被截断，只能显示最后 5000 行内容。
car  :  97.06936478614807  :  [532, 351, 564, 374]
= 1
person  :  59.58234667778015  :  [577, 342, 586, 366]
+ 2.0
person  :  94.83361840248108  :  [406, 345, 418, 383]
+ 3.0
person  :  30.277106165885925  :  [414, 349, 424, 380]
+ 4.0
----_90.jpg-----
car  :  98.1653094291687  :  [229, 390, 436, 527]
+ 2.0
car  :  99.26069974899292  :  [467, 484, 635, 665]
+ 3.0
car  :  49.374085664749146  :  [3, 321, 10, 331]
+ 4.0
person  :  32.35735297203064  :  [94, 324, 105, 344]
+ 5.0
person  :  41.077154874801636  :  [275, 320, 286, 348]
+ 6.0
fire hydrant  :  49.21405613422394  :  [276, 339, 290, 369]
= 1
car  :  88.02328109741211  :  [68, 340, 106, 372]
+ 5.0
car  :  99.60961937904358  :  [97, 343, 163, 392]
+ 6.0
----_180.jpg-----
car  :  97.82427549362183  :  [0, 420, 257, 636]
+ 7.0
car  :  99.9753475189209  :  [254, 371, 413, 465]
+ 8.0
person  :  91.4517879486084  :  [369, 342, 383, 371]
+ 7.0
person  :  87.87805438041687  :  [388, 343, 400, 375]
+ 8.0
person  

In [ ]:
df.tail()

,Location,Number of private cycles,Number of cycle hire bikes,Total cycles,lat,lng,geometry,LSOA11CD,LSOA11NM,accounting,airport,amusement park,aquarium,art gallery,atm,bakery,bank,bar,beauty salon,bicycle store,book store,bowling alley,bus station,cafe,campground,car dealer,car rental,car repair,car wash,casino,cemetery,church,city hall,clothing store,convenience store,courthouse,dentist,department store,doctor,drugstore,electrician,electronics store,embassy,fire station,florist,funeral home,furniture store,gas station,gym,hair care,hardware store,hindu temple,home goods store,hospital,insurance agency,jewelry store,laundry,lawyer,library,light rail station,liquor store,local government office,locksmith,lodging,meal delivery,meal takeaway,mosque,movie rental,movie theater,moving company,museum,night club,painter,park,parking,pet store,pharmacy,physiotherapist,plumber,police,post office,primary school,real estate agency,restaurant,roofing contractor,rv park,school,secondary school,shoe store,shopping mall,spa,stadium,storage,store,subway station,supermarket,synagogue,taxi stand,tourist attraction,train station,transit station,travel agency,university,veterinary care,zoo,person,bicycle,car,motorcycle,bus,train,truck,traffic light,fire hydrant,stop sign,parking meter,bench,dog
176,Weston Street,465.250,5.000,470.250,51.500540,-0.085180,POINT (-0.0851803 51.5005395),E01032803,Southwark 003J,10,0,2,2,25,19,17,17,47,NaN,NaN,NaN,NaN,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,1,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,7,1,NaN,11,14,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,60,NaN,NaN,15,NaN,NaN,2,NaN,0,NaN,NaN,4,20,NaN,NaN,NaN,2,49,NaN,2,NaN,0,10.0,2.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,Whitehall (south of Horse Guards Avenue),1444.500,162.125,1606.625,51.504944,-0.124818,POINT (-0.1248179 51.5049443),E01004736,Westminster 018C,18,2,0,1,14,8,10,13,40,NaN,NaN,NaN,NaN,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,7,10,NaN,7,13,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,60,NaN,NaN,14,NaN,NaN,4,NaN,0,NaN,NaN,7,10,NaN,NaN,NaN,7,36,NaN,2,NaN,0,40.0,NaN,11.0,NaN,4.0,NaN,1.0,5.0,NaN,NaN,NaN,NaN,NaN
178,Wimpole Street,132.500,14.875,147.375,51.517978,-0.148147,POINT (-0.1481473 51.5179779),E01004713,Westminster 011A,60,1,0,1,51,10,14,26,36,NaN,NaN,NaN,NaN,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,5,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,9,10,NaN,15,33,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,60,NaN,NaN,31,NaN,NaN,7,NaN,1,NaN,NaN,10,13,NaN,NaN,NaN,4,53,NaN,11,NaN,0,2.0,2.0,36.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
179,Wood Street,464.125,76.750,540.875,51.516491,-0.094044,POINT (-0.0940443 51.5164907),E01000002,City of London 001B,58,1,0,0,9,19,7,60,60,NaN,NaN,NaN,NaN,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,9,7,NaN,5,19,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,60,NaN,NaN,11,NaN,NaN,5,NaN,0,NaN,NaN,13,23,NaN,NaN,NaN,12,52,NaN,2,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,York Road,687.625,113.625,801.250,51.502856,-0.115946,POINT (-0.1159456 51.502856),E01032582,Lambeth 036E,7,0,0,1,10,6,7,7,32,NaN,NaN,NaN,NaN,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,2,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,2,3,NaN,6,22,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,60,NaN,NaN,0,NaN,NaN,4,NaN,0,NaN,NaN,8,16,NaN,NaN,NaN,7,47,NaN,2,NaN,0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
name = 'Chapter Kings Cross'
execution_path1 = '/content/drive/MyDrive/Streetview/Outer_GSV_result'
execution_path2 = '/content/drive/MyDrive/Streetview/Outer_GSV_data/' + name
print(execution_path)

/content/drive/MyDrive/Streetview/Chapter Kings Cross


In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path1 , "yolo.h5"))
detector.loadModel()
list = ['0.jpg','90.jpg','180.jpg','270.jpg']
for name in list:
  detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path2 , name), output_image_path=os.path.join(execution_path2 , 'result_'+name), minimum_percentage_probability=30)

  for eachObject in detections:
      print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
  
  print("--------------------------------")

person  :  93.68919730186462  :  [116, 351, 168, 433]
--------------------------------
bicycle  :  70.62271237373352  :  [7, 404, 63, 474]
--------------------------------
bicycle  :  86.78436279296875  :  [23, 395, 106, 474]
--------------------------------
bicycle  :  97.48976826667786  :  [115, 400, 167, 472]
--------------------------------
bicycle  :  42.897605895996094  :  [12, 422, 44, 472]
--------------------------------
bus  :  99.88148808479309  :  [355, 291, 452, 371]
--------------------------------
person  :  35.79421639442444  :  [225, 319, 233, 337]
--------------------------------
bicycle  :  65.03438949584961  :  [105, 344, 131, 378]
--------------------------------
car  :  42.730554938316345  :  [115, 333, 165, 351]
--------------------------------
car  :  96.96707725524902  :  [224, 334, 257, 370]
--------------------------------
bench  :  71.68817520141602  :  [509, 395, 540, 430]
--------------------------------
bench  :  39.14459049701691  :  [604, 421, 638, 467]

In [ ]:
df.to_csv('./drive/MyDrive/Streetview/central_2019_avg_newest.csv', index=False)